In [1]:
import requests
import json
import base64
import os

def send_chat_completion_request(url, api_key, model, payload):
    """
    Sends a POST request to the specified chat completion API endpoint.

    Args:
        url (str): The API endpoint URL.
        api_key (str): The API authorization key.
        model (str): The model to use (e.g., "llava-1.6-mistral").
        payload (dict): The request payload including prompt, image_data, etc.

    Returns:
        dict: The JSON response from the API.

    Raises:
        requests.exceptions.HTTPError: If the HTTP request returned an unsuccessful status code.
        requests.exceptions.RequestException: For other request-related errors.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    print("Sending Payload:")
    print(json.dumps(payload, indent=4))  # Debugging

    try:
        response = requests.post(url, headers=headers, json=payload)
        print(f"Status Code: {response.status_code}")  # Debugging
        print(f"Response Text: {response.text}")     # Debugging
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # HTTP error
        print(f"Response content: {response.text}")
    except requests.exceptions.RequestException as err:
        print(f"Other error occurred: {err}")  # Other errors
    return None


# Define the API endpoint
url = "http://localhost:8080/completion"

# Authorization key (replace 'no-key' with your actual key if needed)
api_key = "no-key"

# Model to use
model = "llava-1.6-mistral"

# Path to your PNG image
image_path = "thiemo-jakob.png"

# Check if the image file exists
if not os.path.isfile(image_path):
    print(f"Image file not found at path: {image_path}")
else:
    # Read and encode the image to base64
    try:
        with open(image_path, "rb") as img_file:
            image_data = img_file.read()
            base64_image = base64.b64encode(image_data).decode('utf-8')
    except Exception as e:
        print(f"Failed to read or encode image: {e}")
        base64_image = None

    if base64_image:
        # Prepare the request payload
        payload = {
            "model": model,
            "prompt": "USER:[img-12] Describe the image and what you see. \nASSISTANT:",  # Modify this prompt as needed
            "image_data": [
                {
                    "data": base64_image,
                    "id": 12  # The ID that matches [img-12] in the prompt
                }
            ],
            "slot_id": -1,  # Optional: Modify if needed
            "cache_prompt": False  # Optional: Use this if needed
        }

        # Send the request
        response = send_chat_completion_request(url, api_key, model, payload)

        if response:
            # Pretty-print the JSON response
            print(json.dumps(response, indent=4))
        else:
            print("Failed to get a valid response from the API.")


Sending Payload:
{
    "model": "llava-1.6-mistral",
    "prompt": "USER:[img-12] Describe the image and what you see. \nASSISTANT:",
    "image_data": [
        {
            "data": "...",
            "id": 12
        }
    ],
    "slot_id": -1,
    "cache_prompt": false
}
Status Code: 200
Response Text: {"content":" The image features a group of people in a stadium, with some of them being men dressed in traditional Arab garb. They are gathered around a group of people who are holding a large book or award, possibly a Guinness World Record certificate. The men in the stadium appear to be posing for a group photo while others are holding flags.\n\nThere are several people standing around the main group, all observing the situation. The stadium has an atmosphere of celebration and camaraderie, as everyone seems to be enjoying the moment.</s>","generation_settings":{"dynatemp_exponent":1.0,"dynatemp_range":0.0,"frequency_penalty":0.0,"grammar":"","ignore_eos":false,"logit_bias":[],"min